# Can Recurrent Neural Networks Warp Time?

> Note: **Python3** is used

In [1]:
import numpy as np

## The dataset 

In [2]:
alphabet = {'dummy': 0, 'signal':1, 'chars':np.arange(2,10)}
print(alphabet, len(alphabet))

{'dummy': 0, 'signal': 1, 'chars': array([2, 3, 4, 5, 6, 7, 8, 9])} 3


In [3]:
def onehot(Y, classes, num_classes=9):
    #z = np.zeros((num_classes, *Y.shape))
    z = np.array([Y==i for i in classes])
    return np.transpose(z, [1,2,0]).astype(np.float)

def dataset(N,T, alphabet):
    in_seq = np.random.choice(np.arange(2,10), size=(N,10), replace=True)
    x = np.append(in_seq, np.zeros((N,T+10)), axis=1)
    x[:, -11] = 1. # signal=1
    y = np.zeros((N,T+20))
    y[:, -10:] = in_seq
    X = onehot(x, np.arange(10), 10)
    Y = onehot(y, np.arange(10), 10)
    return X,Y

In [4]:
N = 50000 # number of training samples
T = 500   # length of sequence=T+20
m = 10    # number of alphabets

In [5]:
x_train, y_train = dataset(N, T, alphabet)

## The model

In [6]:
from tensorflow.keras.layers import LSTM, Input
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras import backend as K

### Chrono Initializer  

In [7]:
# define a custom initializer in keras style
class ChronoInitializer(RandomUniform):
    def __init__(self, Tmax, seed=None):
        super().__init__(minval=1, maxval=Tmax-1, seed=seed)
        self.Tmax = Tmax
        
    def __call__(self, shape, dtype=None, partition_info=None):
        bias = super().__call__(shape=shape, dtype=dtype, partition_info=None)
        return K.log(bias)

In [ ]:
# def chrono_initializer(shape, dtype=None):
#     return K.random_normal(shape, dtype=dtype)

In [8]:
T_max = 3*T/2
chrono_init = ChronoInitializer(Tmax=T_max)

In [12]:
# define the model
inputs = Input(shape=(T+20,m))
lstm = LSTM(units=128, unroll=True, use_bias=True, bias_initializer=chrono_init, implementation=2)
outputs = lstm(inputs)
model = Model(inputs=inputs, outputs=outputs)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 520, 10)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               71168     
Total params: 71,168
Trainable params: 71,168
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=30, validation_split=0.2)